# Warstwy konwolucyjne w sieciach neuronowych 

**Warstwa konwolucyjna** to nic więcej niż transformacja dokonująca operacji konwolucji na wejściowym obrazku. Wyjściem z takiej warstwy jest również obrazek - tyle że najczęściej w zupełnie innej przestrzeni "kolorów" (najczęściej "kolorów" - czyli kanałów jest już dużo więcej niż 3). Wagami danej warstwy są filtry - każda warstwa jest reprezentowana przez zestaw filtrów o tych samych wymiarach.

<img src="Grafika/convolve.png" width="550">

$$
O[u, v] = \sum\limits_{i=1}^{2k+1}\sum\limits_{j=1}^{2k+1} M[u-k+i,v-k+j] \cdot K[i,j]
$$

W naszej warstwie jest jeden filtr o wymiarach $3 \times 3$, a na wejściu dostajemy obrazek $28 \times 28$. Przy użyciu *zero-paddingu* na wyjściu otrzymamy również obrazek o wymiarach $28 \times 28$.  

Gdybyśmy chcieli uzyskać to samo wyjście za pomocą warstwy typu *dense*, to potrzebowalibyśmy połączyć każdy z $28 \cdot 28$ pikseli wejściowych z każdym z $28 \cdot 28$ pikseli wyjściowych. Każde połączenie to jedna liczba rzeczywista, więc do opisania tej transformacji potrzebowalibyśmy $28^4 = 614656$ parametrów.  

Dla porównania - przy użyciu warstwy konwolucyjnej mamy tylko **9 parametrów**.

Ogólne zalety warstwy konwolucyjnej można zapisać w trzech punktach:
* rzadkie oddziaływania (sparse interactions) - połączenia między kolejnymi warstwami nie występują pomiędzy każdym neuronem - każdy piksel wpływa tylko na pewne otoczenie kolejnej warstwy parametrów,  
* współdzielenie parametrów - wielokrotnie stosujemy te same wagi na różnych częściach obrazka. Sprawia to, że model potrzebuje dużo mniej parametrów aby przekazać podobną ilość istotnych informacji,  
* ekwiwariancja względem translacji - jeżeli przesuniemy nasz obrazek o pewną ilość pikseli, to samo przesunięcie zaobserwujemy na warstwie wyjściowej.

**Jak propaguje się gradient w warstwie konwolucyjnej**?

W wyniku transformacji (*forward pass*) z obrazka wejściowego $M$ oraz filtru $K$ otrzymaliśmy wyjście $O$ zgodnie ze wzorem

$$ O = M * K $$

Wymiary macierzy to:
* $O - N \times N$ 
* $M - (N+k) \times (N+k)$  
* $K - (2k+1) \times (2k+1)$  

Dodatkowo, w wyniku propagacji wstecznej otrzymaliśmy już macierz $\frac{\partial{L}}{\partial{O}}$, i chcemy policzyć $\frac{\partial{L}}{\partial{M}}$ oraz $\frac{\partial{L}}{\partial{K}}$

<br>
Zacznijmy od pochodnej $\frac{\partial{L}}{\partial{K}}$. Jest to macierz o wymiarach takich samych jak $K$, czyli $(2k+1) \times (2k+1)$

Wyprowadzimy wzór na pojedynczy element szukanej macierzy, czyli $\frac{\partial{L}}{\partial{K[i,j]}}$
Zauważmy, że aby policzyć to wyrażenie musimy rozbić pochodną $\frac{\partial{L}}{\partial{O}}$ po każdym z elementów jej macierzy i zsumować otrzymane pochodne:

$$
\frac{\partial{L}}{\partial{K[i,j]}} = \sum\limits_{u=1}^{N}\sum\limits_{v=1}^{N} \frac{\partial{L}}{\partial{O[u,v]}} \cdot \frac{\partial{O}[u,v]}{\partial{K[i,j]}}
$$

Na szczęście drugie wyrażenie możemy otrzymać bezpośrednio ze wzoru który wprowadziliśmy wyżej:
$$
\sum\limits_{u=1}^{N}\sum\limits_{v=1}^{N} \frac{\partial{L}}{\partial{O[u,v]}} \cdot \frac{\partial{O}[u,v]}{\partial{K[i,j]}} = 
\sum\limits_{u=1}^{N}\sum\limits_{v=1}^{N} \frac{\partial{L}}{\partial{O[u,v]}} \cdot M[u-k+i,v-k+j] 
$$

Szybka zamiana kosmetyczna...:

$$
\sum\limits_{u=1}^{N}\sum\limits_{v=1}^{N} \frac{\partial{L}}{\partial{O[u,v]}} \cdot M[u-k+i,v-k+j] =
\sum\limits_{u=1}^{N}\sum\limits_{v=1}^{N} \frac{\partial{L}}{\partial{O[u,v]}} \cdot M[i-k+u,j-k+v]
$$

I ostatecznie dostajemy następujące równanie:
$$
\frac{\partial{L}}{\partial{K[i,j]}} =
\sum\limits_{u=1}^{N}\sum\limits_{v=1}^{N} M[i-k+u,j-k+v] \cdot  \frac{\partial{L}}{\partial{O[u,v]}} 
$$

$$
O[u, v] = \sum\limits_{i=1}^{2k+1}\sum\limits_{j=1}^{2k+1} M[u-k+i,v-k+j] \cdot K[i,j]
$$

Wygląda znajomo? Powinno - to wzór na konwolucję, (z pozmienianymi indeksami). Dostaliśmy więc równanie

### $$
\frac{\partial{L}}{\partial{K}} = M * \frac{\partial{L}}{\partial{O}}
$$

<br>
<br>

Pozostaje nam już tylko policzyc pochodną $\frac{\partial{L}}{\partial{M}}$

Żeby ułatwić sobie trochę zadanie, zapiszemy najpierw powyższy wzór na konwolucję w trochę innej formie:

$$
O[u, v] = \sum\limits_{i=-k}^{k}\sum\limits_{j=-k}^{k} M[u+i,v+j] \cdot K[i,j]
$$

Zakładamy więc, że macierz $K$ indeksujemy od $-k$ do $k$, czyli punkt współrzędne $(0,0)$ są jej środkiem.

Zacznijmy więc podobnie jak wcześniej:  

$$
\frac{\partial{L}}{\partial{M[l,m]}} = \sum\limits_{u=1}^{N}\sum\limits_{v=1}^{N} \frac{\partial{L}}{\partial{O[u,v]}} \cdot \frac{\partial{O}[u,v]}{\partial{M[l,m]}}
$$  

Podstawiając pochodną z powyższego wzoru na konwolucję dostajemy:  

$$\sum\limits_{u=1}^{N}\sum\limits_{v=1}^{N} \frac{\partial{L}}{\partial{O[u,v]}} \cdot \frac{\partial{O}[u,v]}{\partial{M[l,m]}} = 
\sum\limits_{u=1}^{N}\sum\limits_{v=1}^{N} \frac{\partial{L}}{\partial{O[u,v]}} \cdot K[l-u, m-v]
$$

Skąd to dziwne indeksowanie? W naszej pochodnej indeksy macierzy $M$ to $l$ i $m$, a we wzorze na konwolucję mamy $u+i$ oraz $v+j$. Wynikiem pochodnej jest wyrażenie $K[i,j]$, więc musimy podstawić coś pod $i$ oraz $j$. Z równań 
$$l = u+i \ ; \ m = v+j$$ dostajemy 
$$i = l-u \ ; \ j=m-v.$$


<br>
Zauważmy, że w tym momencie nasza konwolucja jest "za duża" - tzn wiemy że nasz filtr $K$ ma indeksy od $-k$ do $k$, a indeksy w sumach to $1 \ldots N$. Możemy więc w prosty sposób ograniczyć naszą sumę, wracając przy okazji do indeksowania za pomocą $i,j$:  

$$
\sum\limits_{u=1}^{N}\sum\limits_{v=1}^{N} \frac{\partial{L}}{\partial{O[u,v]}} \cdot K[l-u, m-v] = 
\sum\limits_{i=-k}^{k}\sum\limits_{j=-k}^{k} \frac{\partial{L}}{\partial{O[l-i,m-j]}} \cdot K[i, j] 
$$

Widzimy więc, że otrzymaliśmy **prawie** wzór na konwolucję macierzy $\frac{\partial{L}}{\partial{O}}$ z macierzą $K$. Jedyne, co się nie zgadza, to indeksy w pierwszym wyrażeniu - mamy odpowiednio $l-i$ oraz $m-j$, a chcielibyśmy mieć $l+i$ oraz $m+j$.  

Ponieważ jednak nasze sumy są symetryczne, to możemy po prostu zamienić każde $i$ na $-i$, i analogicznie każde $j$ na $-j$:  

$$
\sum\limits_{i=-k}^{k}\sum\limits_{j=-k}^{k} \frac{\partial{L}}{\partial{O[l-i,m-j]}} \cdot K[i, j] =
\sum\limits_{i=-k}^{k}\sum\limits_{j=-k}^{k} \frac{\partial{L}}{\partial{O[l+i,m+j]}} \cdot K[-i,-j] 
$$


Pozornie zamieniliśmy siekierkę na kijek - ale jeśli się zastanowimy nad wyrażeniem $K[-i,-j]$, to możemy o nim myśleć jako o współrzędnej $(i,j)$ macierzy $K$ obróconej o 180 stopni!  

Ostatecznie więc dostajemy:  

$$
\frac{\partial{L}}{\partial{M[l,m]}} = 
\sum\limits_{i=-k}^{k}\sum\limits_{j=-k}^{k} \frac{\partial{L}}{\partial{O[l+i,m+j]}} \cdot K[-i,-j] 
$$

co tłumaczy się na 

### $$
\frac{\partial{L}}{\partial{M}} = \frac{\partial{L}}{\partial{O}} * K_{rot180}
$$

### Pooling

Użycie warstwy gęstej na tak złożonym wejściu prowadzi do ogromnej ilości perametrów. Pojawia się potrzeba wprowadzenia mechanizmu, który będzie redukował wymiarowość obrazka przy jednoczesnym zachowaniu jak największej ilości obserwacji.

Odpowiedzią na ten problem jest tzw. **pooling**. Stosuje się głównie dwa rodzaje poolingu - *max pooling* i *average pooling*. Każda z tych operacji przechodzi po obrazku i przeprowadza odpowiednią agregację na jego segmentach. 

#### Wizualizacja

![](Grafika/pooling.JPG)

W przypadku *average poolingu* gradient jest propagowany po równo na każdy piksel w małym segmencie. Dla *max poolingu* cały gradient przechodzi na ten piksel, który miał w warstwie wejściowej maksymalną wartość. 